# Size a HPP plant based on a simplified hpp model

This **static** notebook illustrates how to solve hybrid power plant sizing optimization in a specific location based on preselected turbine type.

**To execute this setup a server is required as it relies on paralle evaluation of the model.**

Sizing a hybrid power plant consists on designing the following parameters:

### Design Variables

**Wind Plant design:**

- Number of wind turbines in the wind plant [-] (`Nwt`)
- Wind power installation density [MW/km2] (`wind_MW_per_km2`): This parameter controls how closely spaced are the turbines, which in turns affect how much wake losses are present.

**PV Plant design:**

- Solar plant power capacity [MW] (`solar_MW`)

**Battery Storage design:**

- Battery power [MW] (`b_P`)
- Battery energy capacity in hours [MWh] (`b_E_h `): Battery storage capacity in hours of full battery power (`b_E = b_E_h * b_P `). 
- Cost of battery power fluctuations in peak price ratio [-] (`cost_of_batt_degr`): This parameter controls how much penalty is given to do ramps in battery power in the HPP operation.


### Possible objective functions

- LCOE: Levelized cost of energy.
- IRR: Internal rate of return. It is not defined for projects that produce negative Net present values. Hydesigb returns IRR = 0 if the NPV < 0. Nevertheless, optimizations can be problematic for sites without a clear case. 
- NPV/CAPEX: Net present value over total CAPEX. A good proxy variable, that will produce optimal sites with the optimal IRR, but that is defined on sites with negative NPV.

The available variables for optimiaztion are:

```
 'NPV_over_CAPEX',
 'NPV [MEuro]',
 'IRR',
 'LCOE [Euro/MWh]',
 'CAPEX [MEuro]',
 'OPEX [MEuro]',
 'penalty lifetime [MEuro]',
 ```
 

In [1]:
# Install hydesign if needed
import importlib
if not importlib.util.find_spec("hydesign"):
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git    

In [2]:
import pandas as pd
from hydesign.EGO_surrogate_based_optimization_P2X import EGO_path
from hydesign.examples import examples_filepath

# Optimize using EGO

In [3]:
EGO_P2X = f'{EGO_path}EGO_surrogate_based_optimization_P2X.py'

In [4]:
%run $EGO_P2X --help

usage: EGO_surrogate_based_optimization_P2X.py [-h] [--example EXAMPLE]
                                               [--name NAME]
                                               [--longitude LONGITUDE]
                                               [--latitude LATITUDE]
                                               [--altitude ALTITUDE]
                                               [--input_ts_fn INPUT_TS_FN]
                                               [--sim_pars_fn SIM_PARS_FN]
                                               [--opt_var OPT_VAR]
                                               [--num_batteries NUM_BATTERIES]
                                               [--n_procs N_PROCS]
                                               [--n_doe N_DOE]
                                               [--n_clusters N_CLUSTERS]
                                               [--n_seed N_SEED]
                                               [--max_iter MAX_ITER]
                         

#### 

In [5]:
%run $EGO_P2X \
    --example 3 \
    --opt_var "NPV_over_CAPEX"\
    --num_batteries 1\
    --n_procs 31\
    --n_doe 31\
    --n_clusters 15\
    --n_seed 0\
    --max_iter 10\
    --final_design_fn 'hydesign_P2X_output_3.csv'

Selected example site:
---------------------------------------------------
case                                          Europe
name                               France_good_solar
longitude                                   4.229736
latitude                                   44.422011
altitude                                       204.0
input_ts_fn    Europe/input_ts_France_good_solar.csv
sim_pars_fn                      Europe/hpp_pars.yml
price_fn                    Europe/2030-EL_PRICE.csv
price_col                                       FR_R
Name: 3, dtype: object




Sizing a HPP plant at France_good_solar:

longitude = 4.229736
latitude = 44.422011
altitude = 204.0



x=[, 37., 246., 9., 436., 7.36703619, 140., 35.04124764, 161.18996052, 1.96593067, 83., 1., 1.94782122, 30., ]x=[, 50., 345., 9., 209., 7.26227982, 154., 3.8790897, 181.5151681, 1.00188822, 68., 7., 12.85067513, 20., ]x=[, 26., 235., 4., 129., 6.88870481, 309., 42.45617667, 175.28429763, 1.71701169, 5., 3., 5.996178

TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'

In [ ]:
%run $EGO_simple \
    --example 0 \
    --opt_var "NPV_over_CAPEX"\
    --rotor_diameter_m 100\
    --hub_height_m 120\
    --wt_rated_power_MW 2\
    --surface_tilt_deg 20\
    --surface_azimuth_deg 180\
    --DC_AC_ratio 1\
    --num_batteries 2\
    --n_procs  31\
    --n_doe 31\
    --n_clusters 15\
    --n_seed 0\
    --max_iter 10\
    --final_design_fn 'hydesign_simple_design_0.csv'

# Size a HPP plant based on a detailed hpp model

This **static** notebook illustrates how to solve hybrid power plant sizing optimization in a specific location based on preselected turbine type.

**To execute this setup a server is required as it relies on paralle evaluation of the model.**

Sizing a hybrid power plant consists on designing the following parameters:

**Wind Turbine design:**

1. Clearance [m] (`clearance`): Height from the ground to rotor tip at lowest posstion. This parameter controls the wind turbine hub height given a rotor radius: `clearance =  HH - R`. 
2. Specific power of the wind turbine [MW/m2] (`sp`): Defined as `sp = p_rated / ( pi * R^2 )`, it controls the turbine rotor size for a given rated power. Turbines with lower specific power produce more power at lower wind speeds, but are more expensive.
3. Rated power of the wind turbine [MW] (`p_rated`)

**Wind Plant design:**

4. Number of wind turbines in the wind plant [-] (`Nwt`)
5. Wind power installation density [MW/km2] (`wind_MW_per_km2`): This parameter controls how closely spaced are the turbines, which in turns affect how much wake losses are present.

**PV Plant design:**

6. Solar plant power capacity [MW] (`solar_MW`)
7. Surface tilt [deg] (`surface_tilt`)
8. Surface azimuth [deg] (`surface_azimuth`)
9. DC-AC ratio [-] (`solar_DCAC`): This parameter controls how much over-planting of PV (in DC power) is connected to the inverters. It is common practice in PV design to have `solar_DCAC = 1.5`. 

**Battery Storage design:**

10. Battery power [MW] (`b_P`)
11. Battery energy capacity in hours [MWh] (`b_E_h `): Battery storage capacity in hours of full battery power (`b_E = b_E_h * b_P `). 
12. Cost of battery power fluctuations in peak price ratio [-] (`cost_of_batt_degr`): This parameter controls how much penalty is given to do ramps in battery power in the HPP operation.


### Possible objective functions

- LCOE: Levelized cost of energy.
- IRR: Internal rate of return. It is not defined for projects that produce negative Net present values. Hydesigb returns IRR = 0 if the NPV < 0. Nevertheless, optimizations can be problematic for sites without a clear case. 
- NPV/CAPEX: Net present value over total CAPEX. A good proxy variable, that will produce optimal sites with the optimal IRR, but that is defined on sites with negative NPV.

The available variables for optimiaztion are:

```
 'NPV_over_CAPEX',
 'NPV [MEuro]',
 'IRR',
 'LCOE [Euro/MWh]',
 'CAPEX [MEuro]',
 'OPEX [MEuro]',
 'penalty lifetime [MEuro]',
 ```
 

In [4]:
EGO_full = f'{EGO_path}EGO_surrogate_based_optimization_P2X.py'

In [5]:
%run $EGO_full --help

usage: EGO_surrogate_based_optimization_P2X.py [-h] [--example EXAMPLE]
                                               [--name NAME]
                                               [--longitude LONGITUDE]
                                               [--latitude LATITUDE]
                                               [--altitude ALTITUDE]
                                               [--input_ts_fn INPUT_TS_FN]
                                               [--sim_pars_fn SIM_PARS_FN]
                                               [--opt_var OPT_VAR]
                                               [--num_batteries NUM_BATTERIES]
                                               [--n_procs N_PROCS]
                                               [--n_doe N_DOE]
                                               [--n_clusters N_CLUSTERS]
                                               [--n_seed N_SEED]
                                               [--max_iter MAX_ITER]
                         

In [6]:
%run $EGO_full \
    --example 0 \
    --opt_var "NPV_over_CAPEX"\
    --num_batteries 1\
    --n_procs  1\
    --n_doe 31\
    --n_clusters 2\
    --n_seed 0\
    --max_iter 10\
    --final_design_fn 'hydesign_design_0.csv'

Selected example site:
---------------------------------------------------
case                                              India
name                              Indian_site_good_wind
longitude                                     77.500226
latitude                                       8.334294
altitude                                     679.803454
input_ts_fn    India/input_ts_Indian_site_good_wind.csv
sim_pars_fn                          India/hpp_pars.yml
price_fn                  India/Indian_elec_price_t.csv
price_col                                         Price
Name: 0, dtype: object




Sizing a HPP plant at Indian_site_good_wind:

longitude = 77.50022582725498
latitude = 8.334293917013909
altitude = 679.8034540123396


KeyError: 'electrolyzer_capex_cost'